
<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_naranja@4x.png?raw=true" alt="esquema" />
</div>


# Laboratorio de Sistemas de Recomendación

## Objetivos del Laboratorio

1.	Implementar y comparar diferentes tipos de sistemas de recomendación utilizando datos reales.

2.	Explorar el impacto de las características de los datos en la calidad de las recomendaciones.

3.	Desarrollar habilidades en preprocesamiento, modelado y evaluación de sistemas de recomendación.


## Los problemas

A lo largo de este laboratorio deberás enfrentarte a distintos casos prácticos:

- **Caso 1: Sistema de Recomendación Basado en Contenido**

	Debes construir un sistema que recomiende videojuegos similares basándose en su descripción y géneros. Para eso deberás:

	- Preprocesar las descripciones de los juegos (texto) utilizando técnicas de NLP aprendidas durante la clase.

	- Extraer y limpiar las etiquetas de género para su uso en el sistema de recomendación.

	- Implementar una métrica de similitud (como el coseno) para recomendar juegos similares.

	- Evaluar las recomendaciones generadas para un conjunto de juegos específicos.

- **Caso 2: Sistema de Recomendación Basado en Popularidad**

	Debes un sistema que recomiende los juegos más populares según su calificación general y número de reseñas. Para eso deberás:
	
	- Limpiar y convertir las columnas relacionadas con las calificaciones y las reseñas en datos numéricos.
	
	- Identificar los juegos más populares utilizando métricas como la mediana de calificaciones ponderadas.
	
	- Crear una tabla de recomendaciones generales para diferentes géneros.

## Los Datos


- **1. games_description.csv**: Este archivo contiene información detallada sobre videojuegos disponibles en una tienda. Las columnas que tenemos son: 


	- **`name`**: Nombre del videojuego.

	- **`short_description`**: Breve descripción del videojuego.

	- **`long_description`**: Descripción completa y detallada del videojuego.

	- **`genres`**: Lista de géneros asociados al videojuego (por ejemplo, Acción, RPG).

	- **`minimum_system_requirement`**: Requisitos mínimos para ejecutar el videojuego.

	- **`recommend_system_requirement`**: Requisitos recomendados para ejecutar el videojuego.

	- **`release_date`**: Fecha de lanzamiento del videojuego.

	- **`developer`**: Nombre del desarrollador del videojuego.

	- **`publisher`**: Nombre del publicador del videojuego.

	- **`overall_player_rating`**: Calificación general de los jugadores (categórica).

	- **`number_of_reviews_from_purchased_people`**: Número de reseñas realizadas por personas que compraron el juego.

	- **`number_of_english_reviews`**: Número de reseñas en inglés.

	- **`link`**: Enlace al videojuego en la tienda.


- **2. games_ranking.csv**: Este archivo contiene información sobre el ranking de videojuegos basados en diferentes métricas. Las columnsa que tenemos son:


	- **`game_name`**: Nombre del videojuego.

	- **`genre`**: Género principal del videojuego.

	- **`rank_type`**: Tipo de ranking (por ejemplo, ventas, popularidad).

	- **`rank`**: Posición del videojuego en el ranking correspondiente.

----

- Los datos del archivo `games_description.csv` pueden utilizarse para construir sistemas de recomendación basados en contenido.
- Los datos del archivo `games_ranking.csv` son ideales para implementar sistemas basados en popularidad.


In [391]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
pd.set_option("display.max_columns", None)
import numpy as np

# Otros objetivos
# -----------------------------------------------------------------------
import sys

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

# Ruta relativa desde notebooks/1-Preprocesing/ a src/
src_path = "../src/"
sys.path.append(src_path)
import support_preprocesing as sp 
import support_sistemas_recomendacion as ssr 
%load_ext autoreload
%autoreload 2

# Configuración warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [392]:
games_ranking = pd.read_csv("../data/games_ranking.csv")

games_ranking.sort_values("game_name")

,game_name,genre,rank_type,rank
642,69 Balls,Sports & Racing,Review,11
407,7 Days to Die,Simulation,Review,16
391,7 Days to Die,Simulation,Revenue,40
184,ARK: Survival Ascended,Adventure,Revenue,29
106,ARK: Survival Evolved,Action,Review,31
...,...,...,...,...
389,shapez 2,Simulation,Revenue,38
426,theHunter: Call of the Wild™,Simulation,Review,35
382,theHunter: Call of the Wild™,Simulation,Revenue,31
337,theHunter: Call of the Wild™,Simulation,Sales,26


In [393]:
games_ranking.shape

(672, 4)

In [394]:
games_description = pd.read_csv("../data/games_description.csv")
games_description.head(2)

,name,short_description,long_description,genres,minimum_system_requirement,recommend_system_requirement,release_date,developer,publisher,overall_player_rating,number_of_reviews_from_purchased_people,number_of_english_reviews,link
0,Black Myth: Wukong,Black Myth: Wukong is an action RPG rooted in ...,About This Game\n\t\t\t\t\t\t\tBlack Myth: Wuk...,"['Mythology', 'Action RPG', 'Action', 'RPG', '...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"19 Aug, 2024",['Game Science'],['Game Science'],Overwhelmingly Positive,"(654,820)","51,931",https://store.steampowered.com/app/2358720/Bla...
1,Counter-Strike 2,"For over two decades, Counter-Strike has offer...",About This Game\n\t\t\t\t\t\t\tFor over two de...,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...","['OS: Windows® 10', 'Processor: 4 hardware CPU...","['OS: Windows® 10', 'Processor: 4 hardware CPU...","21 Aug, 2012",['Valve'],['Valve'],Very Positive,"(8,313,603)","2,258,990",https://store.steampowered.com/app/730/Counter...


In [395]:
games_description.shape

(290, 13)

In [396]:
sp.exploracion_basica_dataframe(games_description)

El número de datos es 290 y el de columnas es 13

 ------------------------------- 

Los duplicados que tenemos en el conjunto de datos son: 0

 ------------------------------- 

Los unicos que tenemos en el conjunto de datos son:


,count,% unique
name,290,100.00
long_description,290,100.00
link,290,100.00
number_of_english_reviews,289,99.66
genres,288,99.31
number_of_reviews_from_purchased_people,287,98.97
minimum_system_requirement,279,96.21
short_description,277,95.52
release_date,271,93.45
recommend_system_requirement,265,91.38


Las columnas con más del 50% de valores unicos son:
name
long_description
link
number_of_english_reviews
genres
number_of_reviews_from_purchased_people
minimum_system_requirement
short_description
release_date
recommend_system_requirement
developer
publisher

 ------------------------------- 

Las columnas con solo 1 valor único son:

 ------------------------------- 

Los nulos que tenemos en el conjunto de datos son:


,count,% nulos
short_description,13,4.5


Las columnas con más del 50% de valores nulos son:

 ------------------------------- 

Los tipos de las columnas son:


...


 ------------------------------- 

Los valores que tenemos para las columnas categóricas son: 


,columna
0,name
1,short_description
2,long_description
3,genres
4,minimum_system_requirement
5,recommend_system_requirement
6,release_date
7,developer
8,publisher
9,overall_player_rating



 ------------------------------- 

Los valores que tenemos para las columnas numéricas son: 


,columna



 ------------------------------- 



- **Caso 1: Sistema de Recomendación Basado en Contenido**

	Debes construir un sistema que recomiende videojuegos similares basándose en su descripción y géneros. Para eso deberás:

	- Preprocesar las descripciones de los juegos (texto) utilizando técnicas de NLP aprendidas durante la clase.

	- Extraer y limpiar las etiquetas de género para su uso en el sistema de recomendación.

	- Implementar una métrica de similitud (como el coseno) para recomendar juegos similares.

	- Evaluar las recomendaciones generadas para un conjunto de juegos específicos.



In [397]:
# Paso 1: Preprocesar las descripciones
# Función para limpiar y normalizar texto
def preprocess_text(text):
    if not isinstance(text, str):  # Si el valor no es una cadena, lo tratamos como vacío
        return ""
    text = re.sub(r'\n|\t|About This Game', '', text)  # Quitar saltos de línea y encabezados
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Eliminar caracteres especiales
    return text.lower()  # Convertir a minúsculas

# Aplicar la limpieza de texto a la columna 'long_description'
games_description['cleaned_long_description'] = games_description['long_description'].apply(preprocess_text)
games_description['cleaned_short_description'] = games_description['short_description'].apply(preprocess_text)

In [398]:
games_description.head()

,name,short_description,long_description,genres,minimum_system_requirement,recommend_system_requirement,release_date,developer,publisher,overall_player_rating,number_of_reviews_from_purchased_people,number_of_english_reviews,link,cleaned_long_description,cleaned_short_description
0,Black Myth: Wukong,Black Myth: Wukong is an action RPG rooted in ...,About This Game\n\t\t\t\t\t\t\tBlack Myth: Wuk...,"['Mythology', 'Action RPG', 'Action', 'RPG', '...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"19 Aug, 2024",['Game Science'],['Game Science'],Overwhelmingly Positive,"(654,820)","51,931",https://store.steampowered.com/app/2358720/Bla...,black myth wukong is an action rpg rooted in c...,black myth wukong is an action rpg rooted in c...
1,Counter-Strike 2,"For over two decades, Counter-Strike has offer...",About This Game\n\t\t\t\t\t\t\tFor over two de...,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...","['OS: Windows® 10', 'Processor: 4 hardware CPU...","['OS: Windows® 10', 'Processor: 4 hardware CPU...","21 Aug, 2012",['Valve'],['Valve'],Very Positive,"(8,313,603)","2,258,990",https://store.steampowered.com/app/730/Counter...,for over two decades counterstrike has offered...,for over two decades counterstrike has offered...
2,"Warhammer 40,000: Space Marine 2",Embody the superhuman skill and brutality of a...,About This Game\nEmbody the superhuman skill a...,"['Warhammer 40K', 'Action', 'Third-Person Shoo...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"9 Sep, 2024",['Saber Interactive'],['Focus Entertainment'],Very Positive,"(81% of 62,791) All Time","51,920",https://store.steampowered.com/app/2183900/War...,embody the superhuman skill and brutality of a...,embody the superhuman skill and brutality of a...
3,Cyberpunk 2077,"Cyberpunk 2077 is an open-world, action-advent...",About This Game\nCyberpunk 2077 is an open-wor...,"['Cyberpunk', 'Open World', 'Nudity', 'RPG', '...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"10 Dec, 2020",['CD PROJEKT RED'],['CD PROJEKT RED'],Very Positive,"(680,264)","324,124",https://store.steampowered.com/app/1091500/Cyb...,cyberpunk 2077 is an openworld actionadventure...,cyberpunk 2077 is an openworld actionadventure...
4,ELDEN RING,THE CRITICALLY ACCLAIMED FANTASY ACTION RPG. R...,About This Game\nTHE CRITICALLY ACCLAIMED FANT...,"['Souls-like', 'Dark Fantasy', 'Open World', '...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"25 Feb, 2022","['FromSoftware, Inc.']","['FromSoftware, Inc.', 'Bandai Namco Entertain...",Very Positive,"(705,261)","491,741",https://store.steampowered.com/app/1245620/ELD...,the critically acclaimed fantasy action rpgris...,the critically acclaimed fantasy action rpg ri...


In [399]:

# Paso 2: Limpiar y procesar los géneros
# Eliminar caracteres no deseados en los géneros y convertirlos en texto legible
games_description['cleaned_genres'] = games_description['genres'].apply(lambda x: re.sub(r"[^\w\s]", "", x).lower())

In [400]:
# Paso 3: Combinar las características (descripción y géneros)
# Crear una nueva columna combinando la descripción limpia y los géneros
games_description['combined_features'] = games_description['cleaned_short_description'] + " " + games_description['cleaned_genres']

In [401]:
# Paso 4: Vectorización usando CountVectorizer
# Convertir texto a una representación numérica basada en conteos
count_vectorizer = CountVectorizer(stop_words='english', max_features=5000)
count_matrix = count_vectorizer.fit_transform(games_description['combined_features'])

In [402]:
# Paso 5: Calcular similitud coseno
# Generar la matriz de similitud coseno a partir de la matriz Count
cosine_sim_matrix = cosine_similarity(count_matrix, count_matrix)

In [403]:
# Paso 6: Crear una función para recomendar juegos
def recomendaciones(game_name, cosine_sim_matrix, games_data, top_n=5):
    """
    Dada una matriz de similitud coseno y el nombre de un juego,
    recomienda los juegos más similares junto con sus coeficientes de similitud.
    """
    if game_name not in games_data['name'].values:
        return f"Game '{game_name}' not found in the dataset."

    # Obtener el índice del juego en el DataFrame
    game_index = games_data[games_data['name'] == game_name].index[0]
    
    # Obtener puntuaciones de similitud para todos los juegos
    similarity_scores = list(enumerate(cosine_sim_matrix[game_index]))
    
    # Ordenar los juegos por similitud en orden descendente
    sorted_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    
    # Obtener los índices y coeficientes de los juegos más similares (excluyendo el propio juego)
    similar_games = [(games_data.iloc[i[0]]['name'], float(i[1])) for i in sorted_scores[1:top_n + 1]]
    
    return similar_games

# Paso 7: Probar el sistema con un juego específico

jueguito="God of War"

recommendations = recomendaciones(jueguito, cosine_sim_matrix, games_description, top_n=5)

# Mostrar los resultados
print(f"Juegos similares a '{jueguito}':")
for game in recommendations:
    print(f"- {game}")

Juegos similares a 'God of War':
- ('Hogwarts Legacy', 0.5251073270466075)
- ('Black Myth: Wukong', 0.49888765156985865)
- ('Visions of Mana', 0.490361647617904)
- ('Diablo® IV', 0.47869880130712816)
- ('Monster Hunter: World', 0.47000967108038416)


- **Caso 2: Sistema de Recomendación Basado en Popularidad**

	Debes un sistema que recomiende los juegos más populares según su calificación general y número de reseñas. Para eso deberás:
	
	- Limpiar y convertir las columnas relacionadas con las calificaciones y las reseñas en datos numéricos.
	
	- Identificar los juegos más populares utilizando métricas como la mediana de calificaciones ponderadas.
	
	- Crear una tabla de recomendaciones generales para diferentes géneros.

In [404]:
# Paso 1: Unir los datasets en base al nombre del juego
merged_data = pd.merge(
    games_ranking,
    games_description,
    left_on='game_name',  # Nombre del juego en el primer dataset
    right_on='name',  # Nombre del juego en el segundo dataset
    how='inner'  # Solo juegos presentes en ambos datasets
)
merged_data.head()

,game_name,genre,rank_type,rank,name,short_description,long_description,genres,minimum_system_requirement,recommend_system_requirement,release_date,developer,publisher,overall_player_rating,number_of_reviews_from_purchased_people,number_of_english_reviews,link,cleaned_long_description,cleaned_short_description,cleaned_genres,combined_features
0,Counter-Strike 2,Action,Sales,1,Counter-Strike 2,"For over two decades, Counter-Strike has offer...",About This Game\n\t\t\t\t\t\t\tFor over two de...,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...","['OS: Windows® 10', 'Processor: 4 hardware CPU...","['OS: Windows® 10', 'Processor: 4 hardware CPU...","21 Aug, 2012",['Valve'],['Valve'],Very Positive,"(8,313,603)","2,258,990",https://store.steampowered.com/app/730/Counter...,for over two decades counterstrike has offered...,for over two decades counterstrike has offered...,fps shooter multiplayer competitive action tea...,for over two decades counterstrike has offered...
1,"Warhammer 40,000: Space Marine 2",Action,Sales,2,"Warhammer 40,000: Space Marine 2",Embody the superhuman skill and brutality of a...,About This Game\nEmbody the superhuman skill a...,"['Warhammer 40K', 'Action', 'Third-Person Shoo...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"9 Sep, 2024",['Saber Interactive'],['Focus Entertainment'],Very Positive,"(81% of 62,791) All Time","51,920",https://store.steampowered.com/app/2183900/War...,embody the superhuman skill and brutality of a...,embody the superhuman skill and brutality of a...,warhammer 40k action thirdperson shooter adven...,embody the superhuman skill and brutality of a...
2,Cyberpunk 2077,Action,Sales,3,Cyberpunk 2077,"Cyberpunk 2077 is an open-world, action-advent...",About This Game\nCyberpunk 2077 is an open-wor...,"['Cyberpunk', 'Open World', 'Nudity', 'RPG', '...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"10 Dec, 2020",['CD PROJEKT RED'],['CD PROJEKT RED'],Very Positive,"(680,264)","324,124",https://store.steampowered.com/app/1091500/Cyb...,cyberpunk 2077 is an openworld actionadventure...,cyberpunk 2077 is an openworld actionadventure...,cyberpunk open world nudity rpg singleplayer s...,cyberpunk 2077 is an openworld actionadventure...
3,Black Myth: Wukong,Action,Sales,4,Black Myth: Wukong,Black Myth: Wukong is an action RPG rooted in ...,About This Game\n\t\t\t\t\t\t\tBlack Myth: Wuk...,"['Mythology', 'Action RPG', 'Action', 'RPG', '...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"19 Aug, 2024",['Game Science'],['Game Science'],Overwhelmingly Positive,"(654,820)","51,931",https://store.steampowered.com/app/2358720/Bla...,black myth wukong is an action rpg rooted in c...,black myth wukong is an action rpg rooted in c...,mythology action rpg action rpg soulslike comb...,black myth wukong is an action rpg rooted in c...
4,ELDEN RING,Action,Sales,5,ELDEN RING,THE CRITICALLY ACCLAIMED FANTASY ACTION RPG. R...,About This Game\nTHE CRITICALLY ACCLAIMED FANT...,"['Souls-like', 'Dark Fantasy', 'Open World', '...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"25 Feb, 2022","['FromSoftware, Inc.']","['FromSoftware, Inc.', 'Bandai Namco Entertain...",Very Positive,"(705,261)","491,741",https://store.steampowered.com/app/1245620/ELD...,the critically acclaimed fantasy action rpgris...,the critically acclaimed fantasy action rpg ri...,soulslike dark fantasy open world rpg difficul...,the critically acclaimed fantasy action rpg ri...


In [405]:
merged_data.shape

(656, 21)

In [406]:
# Paso 2: Limpiar y preparar columnas adicionales para la recomendación
# Convertir las calificaciones y las reseñas en numéricos
merged_data['number_of_reviews'] = merged_data['number_of_reviews_from_purchased_people'].str.extract(r'(\d+)').astype(float)
rating_map = {
    'Overwhelmingly Positive': 5,
    'Very Positive': 4,
    'Positive': 3,
    'Mixed': 2,
    'Negative': 1,
    'Overwhelmingly Negative': 0
}
merged_data['rating_score'] = merged_data['overall_player_rating'].map(rating_map)

# Calcular la puntuación ponderada como en los pasos previos
C = merged_data['rating_score'].mean()
m = merged_data['number_of_reviews'].median()

def peso_rating(row, m=m, C=C):
    v = row['number_of_reviews']
    R = row['rating_score']
    return (v / (v + m)) * R + (m / (v + m)) * C

merged_data['weighted_score'] = merged_data.apply(peso_rating, axis=1)

merged_data=merged_data.drop(columns=["rank_type","rank"])
merged_data=merged_data.drop_duplicates()
merged_data.sort_values("game_name")

,game_name,genre,name,short_description,long_description,genres,minimum_system_requirement,recommend_system_requirement,release_date,developer,publisher,overall_player_rating,number_of_reviews_from_purchased_people,number_of_english_reviews,link,cleaned_long_description,cleaned_short_description,cleaned_genres,combined_features,number_of_reviews,rating_score,weighted_score
626,69 Balls,Sports & Racing,69 Balls,A Third-Person shooting game but you are not a...,About This Game\nA Third-Person shooting game ...,"['Pool', 'Football (Soccer)', 'Multiplayer', '...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"16 Jun, 2023","['Matthew Choi', 'Plum Point 2']",['GKStudio'],Very Positive,(91% of 143) All Time,120,https://store.steampowered.com/app/2335720/69_...,a thirdperson shooting game but you are not a ...,a thirdperson shooting game but you are not a ...,pool football soccer multiplayer pvp funny tea...,a thirdperson shooting game but you are not a ...,91.0,4.0,4.023298
385,7 Days to Die,Simulation,7 Days to Die,7 Days to Die is an open-world game that is a ...,About This Game\nHOW LONG WILL YOU SURVIVE?Wit...,"['Survival', 'Zombies', 'Multiplayer', 'Open W...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"25 Jul, 2024",['The Fun Pimps'],['The Fun Pimps Entertainment LLC'],Very Positive,"(231,932)","158,982",https://store.steampowered.com/app/251570/7_Da...,how long will you survivewith over 18 million ...,7 days to die is an openworld game that is a u...,survival zombies multiplayer open world open w...,7 days to die is an openworld game that is a u...,231.0,4.0,4.012674
180,ARK: Survival Ascended,Adventure,ARK: Survival Ascended,ARK is reimagined from the ground-up into the ...,About This Game\n\t\t\t\t\t\t\tRespawn into a ...,"['Early Access', 'Survival', 'Open World', 'Di...","['OS: Windows 10/11 with updates', 'Processor:...","['OS: Windows 10/11 with updates', 'Processor:...","25 Oct, 2023","['Studio Wildcard', 'Grove Street Games', 'Ins...","['Studio Wildcard', 'Snail Games USA']",Mixed,"(54,833)","36,814",https://store.steampowered.com/app/2399830/ARK...,respawn into a new dinosaur survival experienc...,ark is reimagined from the groundup into the n...,early access survival open world dinosaurs mul...,ark is reimagined from the groundup into the n...,54.0,2.0,3.199160
103,ARK: Survival Evolved,Action,ARK: Survival Evolved,"Stranded on the shores of a mysterious island,...",About This Game\n\t\t\t\t\t\t\tAs a man or wom...,"['Open World Survival Craft', 'Survival', 'Mul...","['OS *: Windows 7/8.1/10 (64-bit versions)', '...","['OS *: Windows 7/8.1/10 (64-bit versions)', '...","27 Aug, 2017","['Studio Wildcard', 'Instinct Games', 'Efecto ...","['Studio Wildcard', 'Snail Games USA']",Very Positive,"(527,988)","310,038",https://store.steampowered.com/app/346110/ARK_...,as a man or woman stranded naked freezing and ...,stranded on the shores of a mysterious island ...,open world survival craft survival multiplayer...,stranded on the shores of a mysterious island ...,527.0,4.0,4.006452
519,Age of Empires II (Retired),Strategy,Age of Empires II (Retired),Age of Empires II has been re-imagined in high...,About This Game\n\t\t\t\t\t\t\tWith the releas...,"['Strategy', 'RTS', 'Medieval', 'Multiplayer',...","['OS *:Windows Vista, 7, 8 Pro+', 'Processor:1...",['Additional:900x600 minimum display resolution'],"9 Apr, 2013","['Skybox Labs', 'Hidden Path Entertainment', '...",['Xbox Game Studios'],Overwhelmingly Positive,"(82,338)","43,872",https://store.steampowered.com/app/221380/Age_...,with the release of age of empires ii definiti...,age of empires ii has been reimagined in high ...,strategy rts medieval multiplayer classic hist...,age of empires ii has been reimagined in high ...,82.0,5.0,4.543613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543,eFootball™,Sports & Racing,eFoo

In [407]:
# Paso 3: Crear una tabla de los juegos más populares por puntuación ponderada
popular_games = merged_data.sort_values('weighted_score', ascending=False)

In [408]:
merged_data["genre"].unique()

array(['Action', 'Adventure', 'Role-Playing', 'Simulation', 'Strategy',
       'Sports & Racing'], dtype=object)

In [409]:
# Paso 4: Crear recomendaciones generales por género basadas en popularidad

def get_top_games_by_genre(genre, data, top_n=5):
    """
    Devuelve los juegos más populares para un género específico.
    Args:
        genre (str): El género para el cual se buscan los juegos más populares.
        data (DataFrame): El dataset con los datos combinados.
        top_n (int): Número de juegos a devolver.
    Returns:
        DataFrame: Los juegos más populares del género especificado.
    """
    # Filtrar los datos por el género especificado
    genre_filtered = data[data['genre'] == genre]
    
    # Ordenar por la puntuación ponderada
    top_games = genre_filtered.sort_values('weighted_score', ascending=False).head(top_n)
    
    # Retornar los resultados
    return top_games[['name', 'weighted_score', 'rating_score', 'number_of_reviews']]

# Mostrar resultados
print("Juegos más populares según calificación ponderada:")
display(popular_games[['name', 'weighted_score', 'rating_score', 'number_of_reviews']].head(5))

genre_game="Action"
top=get_top_games_by_genre(genre_game,merged_data,top_n=5)
print(f"Juegos más populares de {genre_game}:")
top

Juegos más populares según calificación ponderada:


,name,weighted_score,rating_score,number_of_reviews
386,Garry's Mod,4.929374,5.0,945.0
67,Wallpaper Engine,4.913745,5.0,760.0
638,Driftwood,4.913017,5.0,753.0
29,The Witcher 3: Wild Hunt,4.910088,5.0,726.0
125,The Witcher 3: Wild Hunt,4.910088,5.0,726.0


Juegos más populares de Action:


,name,weighted_score,rating_score,number_of_reviews
67,Wallpaper Engine,4.913745,5.0,760.0
29,The Witcher 3: Wild Hunt,4.910088,5.0,726.0
81,Left 4 Dead 2,4.903339,5.0,670.0
3,Black Myth: Wukong,4.901220,5.0,654.0
88,The Forest,4.868414,5.0,472.0
